# [Projects] 신용거래 이상탐지 데이터

본 프로젝트는 신용카드 거래 이력 데이터를 기반으로

사기 거래 여부를 예측하는 이상탐지 모델을 위한 Feature Engineering을 수행하며, 신용카드 번호 단위로 개인의 행동 패턴을 반영한 파생 피처를 구성해 모델 학습에 적합한 데이터로 정제하는 것을 목표로 진행한다.

프로젝트를 진행하고 나서 보다 유의미한 정보를 도출해내기 위해,
정제된 데이터를 바탕으로 사기 거래만 필터링 후, 주요 파생 피처들의 수치적 분포를 분석하여 사기 거래의 구조적 특성을 도출해 보았다.

## 데이터 불러오기

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
cc_df = pd.read_csv('/content/drive/MyDrive/DS/fraud.csv')

## 데이터 확인하기

해당 데이터셋은 구매 이력에 대한 데이터이다.

- `trans_date_trans_time`: 구매한 날짜와 시간

- `cc_num`: 신용카드 번호
    동일한 값이 빈번하게 등장한다. -> 동일한 신용카드를 사용한 모든 트랜잭션들이 한 줄 한 줄 기록되어 있다.

- `merchant`: 구매 상점 이름

- `category`: 구매 상점의 카테고리

- `amt`: 구매 금액

- `first`/`last`: 이름

- `gende`r: 성별

- `street`/`city`/`state`/`zip`/`lat`/`log`: 고객의 주소 정보

- `city_pop`: zip에 해당하는 인구 수

- `job`: 고객의 직업

- `dob`(day of birth): 생년월일

- `trans_num`: 트랜잭션별 id (필요X)

- `unix time`: trans_date_trans_time의 날짜와 시간을 unix timestamp 형태로 변환시킨 것 (중복된 컬럼)

- `merch_lat`/`merch_long`: 상점의 위도/경도

- `is_fraud`: 사기 거래인지 여부 (예측 대상)


In [8]:
cc_df

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,...,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,...,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,...,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,...,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,fraud_Botsford and Sons,home,134.26,Rebecca,Erickson,F,594 Berry Lights Apt. 392,Wilmington,...,34.2651,-77.8670,186140,English as a second language teacher,1983-02-08,fc860b0d1f89b0b068c9c8db27b6bcc5,1388534208,34.853497,-78.664158,0
491130,2020-12-31 23:56:57,213112402583773,"fraud_Baumbach, Hodkiewicz and Walsh",shopping_pos,25.49,Ana,Howell,F,4664 Sanchez Common Suite 930,Bradley,...,34.0326,-82.2027,1523,Research scientist (physical sciences),1984-06-03,0f0c38fe781b317f733b845c0d6ba448,1388534217,35.008839,-81.475156,0
491131,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
491132,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0


In [9]:
pd.set_option('display.max_columns', 50)

In [10]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [11]:
# 컬럼명과 자료형 확인
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

결측치가 없는 것을 확인할 수 있다.

날짜 관련 컬럼인 trans_date_trans_time과 dob가 object 형태라서,   
해당 컬럼을 사용한다면 datatime 형태로 변환해야 한다.

In [12]:
# 컬럼별 통계량 확인
cc_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000


amt의 max가 25086으로 크지만, 구매 금액인 걸 감안하면 이상치로 판단할 수는 없다.  
또, fraud건 같은 경우는 많은 금액을 빼돌리기 위해서 큰 금액을 결제하는 경향도 있기 때문에 이상치라도 하더라도 나름대로 의미가 있을 수 있다.  
-> amt 컬럼에 대한 이상치는 별도로 처리하지 않겠다.

is_fraud는 0 아니면 1 값인데, min 값을 보면 0.2%로 아주 적은 숫자만 fraud 케이스인 것을 알 수 있다.  
-> imbalanced data(불균형 데이터)라고 한다.

=> True와 False의 비율이 어느 정도 맞아야 예측 모델을 만들었을 때 좋은 결과를 얻기 쉬운데, 해당 데이터셋처럼 한 쪽으로 치우쳐 있는 경우에는 imbalanced data라고 해서 예측하기가 상당히 까다롭다.

## 불필요한 컬럼 제거하기

In [13]:
cc_df.head(3)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0


cc_num(신용카드 번호)는 예측하는 순간에는 필요가 없지만, Feature Engineering을 할 때 동일한 고객(동일한 카드 번호)인지를 확인할 필요가 있다.

merchant는 특정 상점에 fraud 건이 더 빈번하게 발생한다고 생각할 여지가 많지 않다.  
카테고리형 데이터이고, 고윳값들이 너무 많으면 원-핫 인코딩 시 문제가 된다.  
-> drop

first, last의 이름은 필요없는 정보이다.  
-> drop

주소 정보를 담고 있는 lat, long 이외의 나머지 컬럼들은 카테고리 형태이기도 하고 특히 street의 경우에는 매우 다양해서 사실상 의미가 없다.  
-> drop

job도 성별과 마찬가지로 의미가 있을까 생각되지만, 고윳값 개수를 확인해 보면 cc_num과 거의 비슷하다. 즉, 고객 개개인이 다른 직업으로 입력된 것으로 볼 수 있다.  
-> drop

trans_num은 의미가 없는 데이터이다.  
-> drop

unix_time은 trans_date_trans_time과 중복된 정보를 담고 있다.  
-> drop

In [14]:
cc_df['merchant'].nunique()

693

In [15]:
cc_df['job'].nunique()

110

In [16]:
cc_df['cc_num'].nunique()

124

In [17]:
cc_df.drop(['merchant', 'first', 'last', 'street', 'city', 'state', 'zip', 'job', 'trans_num', 'unix_time'], axis=1, inplace=True)

# Feature Engineering

피처 엔지니어링을 할 때는 데이터 자체에 대한 이해도 굉장히 중요하지만, 목적도 충분히 고려해서 방향을 결정해야 한다.

"fraud 케이스가 언제 발생할까? 이걸 어떻게 찾아낼 수 있을까?"

특정 고객의 평소 패턴과 다른 거래가 발생한다면 해당 케이스는 문제가 있을 것 같다고 의심해 볼 수 있다.

In [18]:
cc_df.sort_values('cc_num')

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
454914,2020-12-01 19:01:01,503874407318,health_fitness,46.52,M,29.5894,-98.5201,1595797,1975-12-28,29.784709,-99.107110,0
394566,2020-08-28 08:52:27,503874407318,grocery_pos,93.86,M,29.5894,-98.5201,1595797,1975-12-28,30.284212,-98.681393,0
399198,2020-09-03 14:54:03,503874407318,shopping_net,3.69,M,29.5894,-98.5201,1595797,1975-12-28,30.278887,-98.811829,0
365431,2020-07-20 08:42:31,503874407318,gas_transport,50.12,M,29.5894,-98.5201,1595797,1975-12-28,29.620788,-98.328957,0
391553,2020-08-24 08:43:30,503874407318,misc_net,1.36,M,29.5894,-98.5201,1595797,1975-12-28,29.777531,-97.661993,0
...,...,...,...,...,...,...,...,...,...,...,...,...
218167,2019-12-09 18:09:10,4956828990005111019,misc_pos,1.17,M,40.6747,-74.2239,124967,1980-12-21,40.726319,-73.801472,0
27104,2019-03-01 09:55:44,4956828990005111019,grocery_net,71.09,M,40.6747,-74.2239,124967,1980-12-21,41.614480,-74.157399,0
217757,2019-12-09 13:26:19,4956828990005111019,personal_care,45.12,M,40.6747,-74.2239,124967,1980-12-21,41.567179,-73.811245,0
217797,2019-12-09 13:54:47,4956828990005111019,shopping_pos,7.49,M,40.6747,-74.2239,124967,1980-12-21,41.261494,-73.999228,0


## 1. 구매금액의 z-score 계산하기

구매 금액이 평소보다 훨씬 크면 의심해 볼 수 있다.

단순히 얼마 이상이면 detection을 발동시키는 것이 아니라, 해당 고객의 평소 패턴이 중요하다.

In [19]:
# 해당 cc_num의 모든 거래 건을 추려서 평균과 표준편차를 구한 후,
# 각각의 amt에 대해 연산을 해주면 z-score를 구할 수 있다.

# z-score = (x - 평균) / 표준편차

In [20]:
temp = pd.DataFrame({'a': [10,20,30,20,10,200], 'b': [100,300,200,150,250,200], 'c': [10,500,20,250,25,200]})

In [21]:
temp

,a,b,c
0,10,100,10
1,20,300,500
2,30,200,20
3,20,150,250
4,10,250,25
5,200,200,200


In [22]:
temp.mean()

,0
a,48.333333
b,200.000000
c,167.500000


In [23]:
temp.std()

,0
a,74.677083
b,70.710678
c,192.503247


In [24]:
(temp['a'] - 48.33) / 74.67

,a
0,-0.513325
1,-0.379403
2,-0.245480
3,-0.379403
4,-0.513325
5,2.031204


In [25]:
(temp['b'] - 200) / 70.71

,b
0,-1.414227
1,1.414227
2,0.000000
3,-0.707114
4,0.707114
5,0.000000


In [26]:
(temp['c'] - 167.5) / 192.5

,c
0,-0.818182
1,1.727273
2,-0.766234
3,0.428571
4,-0.740260
5,0.168831


이처럼 z-score를 구하면  
평소의 구매 패턴을 기반으로 각각의 구매 금액에서 평소보다 얼마나 많이 썼는지/덜 썼는지를 나타내 준다.

'z-scroe가 얼마 이상이면 fraud 건일 것이다' 라고 정의할 필요는 없다. 이는 머신러닝 알고리즘에서 자연스럽게 찾아낼 것이다.

In [27]:
cc_df['cc_num'].value_counts()

,count
cc_num,
30270432095985,4392
6538441737335434,4392
4364010865167176,4386
4642255475285942,4386
6538891242532018,4386
...,...
36913587729122,3641
30551643947183,3638
4681601008538160,3638


In [28]:
# 신용카드 번호(cc_num)별 amt의 평균과 표준편차 구하기
# 나중에 DataFrame을 합쳐줘야 하기 때문에 cc_num을 컬럼으로 빼준다.
amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean', 'std']).reset_index()

In [29]:
amt_info

,cc_num,mean,std
0,503874407318,60.253406,127.265783
1,567868110212,83.442558,117.303828
2,571365235126,59.392974,134.289959
3,581686439828,58.578675,149.804992
4,630423337322,56.078113,159.201852
...,...,...,...
119,4792627764422477317,84.135134,107.316736
120,4797297220948468262,56.313583,247.931817
121,4861310130652566408,85.805306,130.998089
122,4906628655840914250,54.243453,154.767184


In [30]:
# cc_num 컬럼을 기준으로, cc_df와 amt_info 데이터를 합치기
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')

In [31]:
cc_df

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,56.078113,159.201852
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,59.858059,132.138802
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,87.328067,113.454416
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,64.317839,174.739042
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,90.747123,165.470881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,87.442772,129.935554
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,58.181297,188.339282
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,50.452289,168.361122
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,88.704297,119.948793


In [32]:
# z-score 구하기
cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [33]:
cc_df

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std,amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,56.078113,159.201852,0.321302
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,59.858059,132.138802,-0.114637
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,87.328067,113.454416,-0.361802
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,64.317839,174.739042,-0.001304
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,90.747123,165.470881,-0.393828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,87.442772,129.935554,0.360311
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,58.181297,188.339282,-0.173577
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,50.452289,168.361122,0.364619
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,88.704297,119.948793,-0.015209


아직은 amt_z가 0에서 크게 떨어지지 않은 일반적인 거래 건들만 보이는데,  
is_fraud가 1인 케이스를 확인해 보자.

In [34]:
cc_df[cc_df['is_fraud'] == 1]

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std,amt_z
4794,2019-01-12 00:59:01,581686439828,gas_transport,11.73,M,41.5205,-80.0573,5507,1973-07-28,41.947427,-79.796264,1,58.578675,149.804992,-0.312731
4816,2019-01-12 03:48:07,581686439828,grocery_pos,328.68,M,41.5205,-80.0573,5507,1973-07-28,42.148618,-79.398595,1,58.578675,149.804992,1.803020
4979,2019-01-12 15:46:10,581686439828,food_dining,120.58,M,41.5205,-80.0573,5507,1973-07-28,42.470024,-80.126576,1,58.578675,149.804992,0.413880
5073,2019-01-12 19:53:59,581686439828,shopping_net,1081.35,M,41.5205,-80.0573,5507,1973-07-28,42.455406,-79.521640,1,58.578675,149.804992,6.827351
5124,2019-01-12 22:44:05,581686439828,shopping_net,776.70,M,41.5205,-80.0573,5507,1973-07-28,40.680209,-79.099101,1,58.578675,149.804992,4.793708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477832,2020-12-21 02:21:41,4716561796955522,grocery_pos,358.24,F,48.2777,-112.8456,743,1972-05-04,47.526202,-113.643313,1,52.537867,106.113023,2.880911
477847,2020-12-21 02:36:03,4716561796955522,shopping_net,859.12,F,48.2777,-112.8456,743,1972-05-04,48.272348,-112.328075,1,52.537867,106.113023,7.601161
479296,2020-12-21 22:38:38,4716561796955522,home,209.84,F,48.2777,-112.8456,743,1972-05-04,49.173669,-112.698767,1,52.537867,106.113023,1.482402
479305,2020-12-21 22:42:11,4716561796955522,food_dining,123.58,F,48.2777,-112.8456,743,1972-05-04,48.913048,-113.214921,1,52.537867,106.113023,0.669495


모든 거래 건들이 그런 건 아니지만, z-score가 크게 나온 건들이 꽤 있다.

-> z-score를 구하는 것이 fraud 케이스를 잡아내는 데 꽤 유용하게 쓰일 수 있을 것 같다.

In [35]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

카테고리까지 고려된 z-score도 구해보자.

In [36]:
# 신용카드 번호(cc_num)별, 카테고리별 amt의 평균과 표준편차 구하기
cat_info = cc_df.groupby(['cc_num', 'category'])['amt'].agg(['mean', 'std']).reset_index()

In [37]:
cat_info

,cc_num,category,mean,std
0,503874407318,entertainment,73.282418,103.050402
1,503874407318,food_dining,38.712305,46.548436
2,503874407318,gas_transport,68.457820,14.730440
3,503874407318,grocery_net,48.931302,18.736252
4,503874407318,grocery_pos,61.987806,23.449569
...,...,...,...,...
1731,4956828990005111019,misc_pos,74.177012,168.341518
1732,4956828990005111019,personal_care,35.379382,44.082579
1733,4956828990005111019,shopping_net,70.019115,239.350164
1734,4956828990005111019,shopping_pos,45.988976,174.986921


In [38]:
# cc_num별로만 합치면, 중복된 cc_num 그리고 다른 카테고리들이 제대로 매칭이 되지 않는다.
cc_df = cc_df.merge(cat_info, on=['cc_num', 'category'], how='left')

In [39]:
cc_df

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,mean,std
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,99.637224,23.904424
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,60.694144,24.513316
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,60.427269,19.558574
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,59.145831,23.345746
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,86.794025,106.330185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,58.958771,51.896818
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,66.667245,319.508780
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,50.923503,52.341751
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,63.856707,52.309370


In [40]:
# z-score 구하기
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

In [41]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

In [42]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698


=> 구매 금액에 대해서는 평소 행동 패턴에 기반하여 금액이 평소보다 얼마나 큰지 이상치를 감지해 내는 두 개의 컬럼 amt_z, cat_amt_z을 구했다.

## 2. 결제 시간 관련 feature 분석

고객마다 신용카드를 빈번하게 사용하는 시간대와, 잘 사용하지 않는 시간대가 있을 것이다.

trans_date_trans_time 컬럼에서 시간만 뽑아내서 각 고객이 주로 어느 시간대에 몇 퍼센트의 비중으로 결제를 하는지, 그리고 각각의 거래 건은 그 중에 어디에 속하는지를 하나의 변수로 추가한다.  
-> 해당 결제 건이 평소에 자주 사용하던 시간대의 결제 건인지, 아니면 좀 드문 시간대에 사용한 결제 건인지도 확인할 수 있다.

=> fraud 케이스를 감지할 수 있는 또 하나의 feature가 될 수 있을 것 같다.

In [43]:
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   category               491134 non-null  object 
 3   amt                    491134 non-null  float64
 4   gender                 491134 non-null  object 
 5   lat                    491134 non-null  float64
 6   long                   491134 non-null  float64
 7   city_pop               491134 non-null  int64  
 8   dob                    491134 non-null  object 
 9   merch_lat              491134 non-null  float64
 10  merch_long             491134 non-null  float64
 11  is_fraud               491134 non-null  int64  
 12  amt_z                  491134 non-null  float64
 13  cat_amt_z              491134 non-null  float64
dtypes: float64(7), int64(3), object(4)
m

In [44]:
cc_df['hour'] = pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour

In [45]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0


신용카드 번호(cc_num)를 기준으로 각 카드가 총 몇 건이 결제가 되었고, 각각 어떤 시간대에 얼마만큼 결제가 되었나를 살펴봐야 한다.

z-score로 구하기에는 제약이 있다.  
예를 들어 23시와 0시는 컴퓨터가 인식하기에는 0과 23만큼의 차이인데, 실제로 시간 단위에서는 한 시간의 차이밖에 안 되는 것이다. 그래서 오히려 같은 시간 범주로 묶어야 되는데 완전히 동떨어진 개념으로 인식하게 된다.

In [46]:
# 시간을 숫자 그대로 두지 않고, 인위적으로 범위를 정해서 나눠준다.
def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

In [47]:
cc_df['hour_cat'] = cc_df['hour'].apply(hour_func)

In [48]:
cc_df['hour_cat'].value_counts()

,count
hour_cat,
afternoon,176801
night,146697
evening,98662
morning,68974


In [49]:
# 신용카드 번호(cc_num)별로 총 몇 건의 결제 건이 있는지 구하기
all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

In [50]:
# 신용카드 번호(cc_num)별, 시간대별로 총 몇 건의 결제 건이 있는지 구하기
hour_cnt = cc_df.groupby(['cc_num', 'hour_cat'])['amt'].count().reset_index()

In [51]:
all_cnt.head()

,cc_num,amt
0,503874407318,3655
1,567868110212,3644
2,571365235126,4374
3,581686439828,3653
4,630423337322,4362


In [52]:
hour_cnt.head()

,cc_num,hour_cat,amt
0,503874407318,afternoon,1280
1,503874407318,evening,737
2,503874407318,morning,558
3,503874407318,night,1080
4,567868110212,afternoon,1228


In [53]:
# 시간대별 퍼센트를 구하기 위해 hour_cnt에 all_cnt를 붙이기
hour_cnt = hour_cnt.merge(all_cnt, on='cc_num', how='left')

In [54]:
hour_cnt.head()

,cc_num,hour_cat,amt_x,amt_y
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [55]:
hour_cnt = hour_cnt.rename({'amt_x': 'hour_cnt', 'amt_y': 'total_cnt'}, axis=1)

In [56]:
hour_cnt.head()

,cc_num,hour_cat,hour_cnt,total_cnt
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [57]:
# total_cnt에 대한 hour_cnt의 퍼센트 구하기
hour_cnt['hour_perc'] = hour_cnt['hour_cnt'] / hour_cnt['total_cnt']

In [58]:
hour_cnt.head(10)

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
0,503874407318,afternoon,1280,3655,0.350205
1,503874407318,evening,737,3655,0.201642
2,503874407318,morning,558,3655,0.152668
3,503874407318,night,1080,3655,0.295486
4,567868110212,afternoon,1228,3644,0.336992
5,567868110212,evening,820,3644,0.225027
6,567868110212,morning,529,3644,0.145170
7,567868110212,night,1067,3644,0.292810
8,571365235126,afternoon,1523,4374,0.348194
9,571365235126,evening,943,4374,0.215592


In [59]:
# 동일한 cc_num에서 hour_perc의 합이 1이 되는지 확인
hour_cnt.loc[0:3]['hour_perc'].sum()

np.float64(1.0)

In [60]:
# 동일한 cc_num에서 hour_perc의 합이 1이 되는지 확인
hour_cnt.groupby('cc_num')['hour_perc'].sum()

,hour_perc
cc_num,
503874407318,1.0
567868110212,1.0
571365235126,1.0
581686439828,1.0
630423337322,1.0
...,...
4792627764422477317,1.0
4797297220948468262,1.0
4861310130652566408,1.0


In [61]:
# hour_perc 컬럼을 원래 데이터프레임 cc_df에 붙이기 위한 컬럼 정리
hour_cnt = hour_cnt[['cc_num', 'hour_cat', 'hour_perc']]

In [62]:
# cc_num, hour_cat 컬럼을 기준으로, cc_df에 hour_cnt를 합치기
cc_df = cc_df.merge(hour_cnt, on=['cc_num', 'hour_cat'], how='left')

In [63]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat,hour_perc
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0,evening,0.196470
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0,evening,0.214383
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0,evening,0.217252
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0,evening,0.213600
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0,evening,0.202882


In [64]:
# 불필요한 컬럼 제거하기
cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis=1, inplace=True)

시간에 대한 정보들을 얻었다.

이외에도,
요일을 찾아서 주말과 평일을 구분해 본다든가

이전 거래와의 시간 혹은 날짜 차이를 구해 본다든가  
-> 카드를 쓰다가 다른 카드르 발급받아서 한동안 사용하지 않을 수 있는데, 만약 세 달 동안 안쓰던 카드가 갑자기 높은 금액으로 결제됐다고 한다면, 이 케이스도 fraud detection해 볼 수 있는 부분이다.

<br>

=> 평소에 많이 거래하는 시간대인지 아닌지를 확인하는 hour_pecr로 생성했다.

## 3. 거리 관련 feature 분석

고객의 위치와 상점의 위치를 사이의 거리를 구한 후 z-score를 사용해서,  
해당 고객의 기존 거래 패턴의 평균과 표준편차를 구한다.  
새로운 거래 건이 등장했을 때, 기존 패턴에서 너무 벗어나는 게 있지 않은가를 알아본다.

In [65]:
!pip install geopy

In [66]:
# 거리 계산 기능을 제공하는 라이브러리 불러오기
from geopy.distance import distance

In [67]:
distance((48.8878, -118.2105), (49.159047, -118.186462)).km

30.216618410409005

In [68]:
# 고객의 위치와 상점의 위치 사이의 거리 계산
cc_df['distance'] = cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis=1)

In [69]:
# [참고] 실행에 소요된 시간 확인해 보기

from datetime import datetime

In [70]:
# start_time = datetime.now()

# 실행 코드 여기에 작성
# cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis=1)

# datetime.now() - start_time

In [71]:
# 신용카드 번호()별 거리의 평균과 표준편차 구하기
dist_info = cc_df.groupby('cc_num')['distance'].agg(['mean', 'std']).reset_index()

In [72]:
cc_df

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,1.450980,0.209201,98.043837
491130,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.128877,0.207534,127.240424
491131,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,1.163822,0.160339,100.023736
491132,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,0.440137,0.218022,80.887812


In [73]:
# cc_df에 dist_info 합치기
cc_df = cc_df.merge(dist_info, on='cc_num', how='left')

In [74]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,mean,std
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117


In [75]:
# z-score 구하기
cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean']) / cc_df['std']

In [76]:
cc_df

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,mean,std,dist_z
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031,0.345370
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606,-0.272693
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493,-1.824486
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117,0.002138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,1.450980,0.209201,98.043837,77.251486,29.270739,0.710346
491130,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.128877,0.207534,127.240424,77.491795,29.899826,1.663843
491131,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,1.163822,0.160339,100.023736,79.172599,30.051452,0.693848
491132,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,0.440137,0.218022,80.887812,72.656420,28.552151,0.288293


z-score가 1.6으로 꽤 높은 편의 케이스를 확인해 보면, distance가 127로 꽤 높은 편이다.  
fraud 건은 아니지만 이런 정보 하나하나가 fraud 건을 찾는 데에 도움이 되기 때문에 의심해 볼만한 여지가 있는 것이다.

In [77]:
# 불필요한 컬럼 제거하기
# distance는 절댓값 자체가 크면 의심의 여지가 있으니 남겨두겠다.
cc_df.drop(['lat', 'long', 'merch_lat', 'merch_long', 'mean', 'std'], axis=1, inplace=True)

지금까지 구매 금액, 구매 시간, 거리에 대해 고객의 평소 구매 패턴과 얼마나 다른지를 확인할 수 있는 feature들을 뽑아보았다.

RFM 기법에서 차용한 방법으로  
구매 시간에 대한(Rencency), 구매 금액에 대한(Monetary), Frequency는 사용지 않았지만 거리에 대한 Feature Engineering을 수행했다.

## 4. 나이 feature 만들어보기

object 타입의 dob 컬럼을 시간 형태로 변경해야 한다.  
태어난 월/일은 그다지 중요하지 않아 보이지만, 연도는 나이와 직결된 부분이기 때문에 나이 개념으로 활용하기 위해 연도만 남겨두겠다.

In [78]:
cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

In [79]:
cc_df

,cc_num,category,amt,gender,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,630423337322,grocery_pos,107.23,F,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370
2,180048185037117,grocery_net,46.28,F,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693
3,374930071163758,grocery_pos,64.09,M,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486
4,2712209726293386,misc_pos,25.58,F,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138
...,...,...,...,...,...,...,...,...,...,...,...,...
491129,6011109736646996,home,134.26,F,186140,1983,0,0.360311,1.450980,0.209201,98.043837,0.710346
491130,213112402583773,shopping_pos,25.49,F,1523,1984,0,-0.173577,-0.128877,0.207534,127.240424,1.663843
491131,3556613125071656,kids_pets,111.84,M,28739,1999,0,0.364619,1.163822,0.160339,100.023736,0.693848
491132,6011724471098086,kids_pets,86.88,F,3684,1981,0,-0.015209,0.440137,0.218022,80.887812,0.288293


## 범주형 데이터의 One-Hot Encoding

In [80]:
cc_df['category'].nunique()

14

In [81]:
# object 타입의 컬럼 category, gender에 대해 원-핫 인코딩
cc_df = pd.get_dummies(cc_df, drop_first=True)

In [82]:
cc_df.head()

,cc_num,amt,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,630423337322,107.23,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,4956828990005111019,44.71,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,180048185037117,46.28,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,374930071163758,64.09,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486,False,False,False,True,False,False,False,False,False,False,False,False,False,True
4,2712209726293386,25.58,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138,False,False,False,False,False,False,False,False,True,False,False,False,False,False


해당 데이터셋을 머신러닝 모델에 학습 시키기 전에, 불필요한 데이터가 있는지 살펴본다.

cc_num은 일종의 id 개념이기 머신러닝 모델 학습에는 필요가 없다. trans_num를 drop한 것과 동일한 이치로 필요가 없지만, 지금까지는 z-score를 구하기 위해 남겨둔 것이다.

In [83]:
cc_df.drop('cc_num', axis=1, inplace=True)

In [84]:
cc_df

,amt,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,107.23,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,44.71,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,46.28,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,64.09,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486,False,False,False,True,False,False,False,False,False,False,False,False,False,True
4,25.58,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138,False,False,False,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491129,134.26,186140,1983,0,0.360311,1.450980,0.209201,98.043837,0.710346,False,False,False,False,False,True,False,False,False,False,False,False,False,False
491130,25.49,1523,1984,0,-0.173577,-0.128877,0.207534,127.240424,1.663843,False,False,False,False,False,False,False,False,False,False,False,True,False,False
491131,111.84,28739,1999,0,0.364619,1.163822,0.160339,100.023736,0.693848,False,False,False,False,False,False,True,False,False,False,False,False,False,True
491132,86.88,3684,1981,0,-0.015209,0.440137,0.218022,80.887812,0.288293,False,False,False,False,False,False,True,False,False,False,False,False,False,False


# 정규화된 파생 피처로 본 사기 거래 건 분석



전체 거래 데이터를 담은 `cc_df`를 기반으로,

`is_fraud == 1`인 사기 거래만 필터링하여 `fraud_df`를 생성하였다.

<br>
사기 거래 건들만 따로 모아 분석한 이유는 다음과 같다.

- 정규화된 파생 피처들(amt_z, cat_amt_z, hour_perc, distance, dist_z)이 전체 거래를 기준으로 만들어졌기 때문에
 → **사기 거래가 이 기준에서 얼마나 벗어나는지** 살펴볼 수 있음

- 이를 통해 머신러닝 모델 학습 전, 정상 거래 대비 사기 거래의 특징 및 분포를 수치로 파악

<br>

분석에 사용된 주요 피처들이다.

`amt_z` : 고객 기준 거래 금액의 z-score.
`cat_amt_z` : 동일 카테고리 내 거래 금액의 z-score  
`hour_perc` : 해당 거래 시간대의 고객 거래 비중  
`distance` : 고객 ↔ 상점 간 거리  
`dist_z` : 고객 ↔ 상점 간 거리 z-score.

참고로, `*_z` 피처들은 전체 사용자나 카테고리 단위로 정규화된 z-score 값이다.



In [138]:
fraud_df = cc_df[cc_df['is_fraud'] == 1]

In [87]:
# 사기 거래만 필터링한 데이터셋 저장
# fraud_df.to_csv('fraud_only.csv', index=False)

In [88]:
# 전체 거래 데이터셋 저장
# cc_df.to_csv('fraud_all.csv', index=False)

In [139]:
fraud_df

,amt,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
4794,11.73,5507,1973,1,-0.312731,-3.859368,0.207501,52.154385,-0.778664,False,True,False,False,False,False,False,False,False,False,False,False,False,True
4816,328.68,5507,1973,1,1.803020,9.831367,0.207501,88.661282,0.497573,False,False,False,True,False,False,False,False,False,False,False,False,False,True
4979,120.58,5507,1973,1,0.413880,1.513892,0.336162,105.622735,1.090525,True,False,False,False,False,False,False,False,False,False,False,False,False,True
5073,1081.35,5507,1973,1,6.827351,2.812675,0.307692,112.931381,1.346027,False,False,False,False,False,False,False,False,False,False,True,False,False,True
5124,776.70,5507,1973,1,4.793708,1.952898,0.307692,123.238036,1.706335,False,False,False,False,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477832,358.24,743,1972,1,2.880911,8.661420,0.204582,102.658922,1.065395,False,False,False,True,False,False,False,False,False,False,False,False,False,False
477847,859.12,743,1972,1,7.601161,3.485939,0.204582,38.419308,-1.195225,False,False,False,False,False,False,False,False,False,False,True,False,False,False
479296,209.84,743,1972,1,1.482402,3.292901,0.299427,100.219589,0.979554,False,False,False,False,False,True,False,False,False,False,False,False,False,False
479305,123.58,743,1972,1,0.669495,1.761706,0.299427,75.721758,0.117465,True,False,False,False,False,False,False,False,False,False,False,False,False,False


In [140]:
# 전체 거래(`491,134건`)의 describe 통계치를 통해 피처의 분포를 확인한다.
cc_df[["amt_z", "cat_amt_z", "hour_perc", "distance", "dist_z"]].describe().round(3)

,amt_z,cat_amt_z,hour_perc,distance,dist_z
count,491134.000,491134.000,491134.000,491134.000,491134.000
mean,0.000,0.000,0.280,76.372,-0.000
std,1.000,0.998,0.082,29.144,1.000
min,-0.923,-4.264,0.091,0.124,-2.730
25%,-0.361,-0.539,0.208,55.604,-0.712
50%,-0.177,-0.229,0.293,78.563,0.079
75%,0.096,0.276,0.349,98.721,0.764
max,63.421,18.900,0.417,149.610,2.323


In [141]:
# 사기 거래(`1244건`)의 describe 통계치를 통해 피처의 분포를 확인한다.
fraud_df[["amt_z", "cat_amt_z", "hour_perc", "distance", "dist_z"]].describe().round(3)

,amt_z,cat_amt_z,hour_perc,distance,dist_z
count,1244.000,1244.000,1244.000,1244.000,1244.000
mean,2.931,3.134,0.228,76.754,0.015
std,3.120,3.468,0.058,28.693,0.983
min,-0.817,-4.264,0.091,1.453,-2.585
25%,-0.188,-0.137,0.201,55.735,-0.704
50%,2.092,3.471,0.209,78.819,0.094
75%,5.381,5.304,0.285,98.947,0.776
max,16.176,16.129,0.417,142.369,2.134


전체 거래에서 사기 거래의 특징을 파악하기 위해,
전체 거래 및 사기 거래의 **통계치를 보여주는두 DataFrame를 비교하기 쉽도록 합쳐보기**로 했다.

각 수치를 보다 직관적으로 비교할 수 있도록, 항목별로 컬럼 순서를 정리해주었다.

In [142]:
# 전체 거래 통계
overall_desc = cc_df[['amt_z', 'cat_amt_z', 'hour_perc', 'distance', 'dist_z']].describe().round(3)

In [143]:
# 사기 거래 통계
fraud_desc = fraud_df[['amt_z', 'cat_amt_z', 'hour_perc', 'distance', 'dist_z']].describe().round(3)

In [144]:
# 행 인덱스를 columns로 바꾸고 transpose
overall_t = overall_desc.T.add_prefix("")
fraud_t = fraud_desc.T.add_prefix("f_")

In [145]:
# 피처명을 열로 복원
overall_t["피처"] = overall_t.index
fraud_t["피처"] = fraud_t.index

In [146]:
# 피처 기준으로 병합
merged = pd.merge(overall_t, fraud_t, on="피처")

In [148]:
# 컬럼 순서 정리
ordered_columns = [
    "피처",
    "mean", "f_mean",
    "std", "f_std",
    "min", "f_min",
    "max", "f_max"
]
merged = merged[ordered_columns]

In [149]:
# 컬럼명 보기 좋게 수정
# f_로 사기 거래 건의 통계치임을 나타냄
merged.columns = ["피처", "평균", "f_평균", "표준편차", "f_표준편차", "최소", "f_최소", "최대", "f_최대"]

In [150]:
merged

,피처,평균,f_평균,표준편차,f_표준편차,최소,f_최소,최대,f_최대
0,amt_z,0.000,2.931,1.000,3.120,-0.923,-0.817,63.421,16.176
1,cat_amt_z,0.000,3.134,0.998,3.468,-4.264,-4.264,18.900,16.129
2,hour_perc,0.280,0.228,0.082,0.058,0.091,0.091,0.417,0.417
3,distance,76.372,76.754,29.144,28.693,0.124,1.453,149.610,142.369
4,dist_z,-0.000,0.015,1.000,0.983,-2.730,-2.585,2.323,2.134


### 1) &nbsp;amt_z: &nbsp;사용자 기준 금액 이상 탐지

In [151]:
# 사기 거래 중 amt_z > 2 인 거래 수
amt_z_outliers = (fraud_df["amt_z"] > 2).sum()

In [152]:
# 전체 사기 거래 수 대비 비율 계산
amt_z_outlier_ratio = round((amt_z_outliers / len(fraud_df)) * 100, 2)

In [153]:
print(f'amt_z > 2인 사기 거래 수: {amt_z_outliers}건')
print(f'전체 사기 거래 수 대비 비율: {amt_z_outlier_ratio}%')

amt_z > 2인 사기 거래 수: 634건
전체 사기 거래 수 대비 비율: 50.96%


사기 거래는 각 고객별로 평균보다 훨씬 높은 금액에서 집중적으로 발생한다.  
단순 고액 결제가 아닌, **해당 고객의 기준에서 이상치인 고액 결제**라는 점이 핵심이다.

### 2) &nbsp;cat_amt_z: &nbsp;카테고리 기준 금액 이상 탐지

동일 업종(카테고리) 내에서도 평소보다 유난히 큰 거래가 사기일 가능성이 높다.

예를 들어, 보통 20달러 쓰는 편의점에서 갑자기 300달러를 결제했다면 이는 `cat_amt_z`가 크게 튀는 사례이다.

### 3) &nbsp;`hour_perc`: &nbsp;고객별 시간대 거래 비중

In [154]:
(fraud_df["hour_perc"] <= 0.2).sum()  # → 292건

np.int64(292)

평균 수치만으로 이상 여부를 판단할 순 없지만, 약 1/4에 해당하는 사기 거래가 **해당 고객이 평소 거의 거래하지 않는 시간대**에서 발생한 것은 의미 있는 신호이다.

### 4) &nbsp;distance: &nbsp;고객 ↔ 상점 간 거리

거리 자체는 단독으로 이상치 탐지의 강한 기준이 되긴 어렵지만, 특정 사용자의 평소 반경에서 벗어났을 경우에는 중요한 단서가 될 수 있다.

### 5) &nbsp;dist_z: &nbsp;사용자 기준 거리 이상도

In [155]:
# 사기 거래 중 dist_z > 2 인 거래 수
dist_z_outliers = (fraud_df["dist_z"] > 2).sum()

In [156]:
# 전체 사기 거래 수 대비 비율 계산
dist_z_outlier_ratio = round((dist_z_outliers / len(fraud_df)) * 100, 2)

In [157]:
print(f'dist_z > 2인 사기 거래 수: {dist_z_outliers}건')
print(f'전체 사기 거래 수 대비 비율: {dist_z_outlier_ratio}%')

dist_z > 2인 사기 거래 수: 7건
전체 사기 거래 수 대비 비율: 0.56%


대부분의 사기 거래는 거리보다는 금액 및 시간대 패턴 이탈에서 두드러지며  
`dist_z`는 일부 사용자에겐 중요한 피처가 될 수 있지만, 전체 기준에서는 약한 편이라는 것을 알 수 있다.

### 추가 인사이트 도출

`amt_z`, `cat_amt_z`, `hour_perc`는 단독으로도 이상 탐지가 강한 피처라는 것을 알 수 있다.

In [158]:
# 세 가지 이상 조건을 동시에 만족하는 사기 거래 수
condition_count = fraud_df[
    (fraud_df["amt_z"] > 2) &
    (fraud_df["cat_amt_z"] > 2) &
    (fraud_df["hour_perc"] <= 0.2)
].shape[0]

In [159]:
# 전체 대비 비율
condition_ratio = round(condition_count / len(fraud_df) * 100, 2)

In [160]:
print(f'조건 만족 거래 수: {condition_count}건 ({condition_ratio}%)')

조건 만족 거래 수: 143건 (11.5%)


이 세 가지 조건을 동시에 만족하는 거래는 전체 사기 거래 중 약 **11.5%**를 차지하는 것으로 나타난다.

이처럼 여러 기준에서 동시에 이상 신호가 나타나는 거래는 단일 조건보다 훨씬 **강한 이상 신호**로 해석할 수 있다.

그렇다면,

_"설명력이 약한 `distance`나 `dist_z` 같은 피처들은 이상 탐지 예측 모델을 위한 변수로는 쓸모가 없는 걸까?"_

<br>

그렇지는 않다.

- 특정 사용자 그룹에서는 해당 변수의 의미가 더 커질 수 있다. 예를 들어, 평소 활동 반경이 좁은 사용자라면 거리가 멀다는 것 자체가 강한 이상 신호가 될 수 있다.

- 또한, “단독으로는 평범해 보이는 distance 값도, 다른 변수와 함께 보면 이상 거래를 식별하는 데 유용한 보조 지표가 될 수 있다.

실제로 머신러닝 모델은 변수 하나만 보는 게 아니라, 여러 변수를 조합해서 판단하기 때문에, 이렇게 단독으로는 약해 보여도 조합에서는 중요한 역할을 하는 경우가 많다. 지금은 `distance`와 `dist_z`는 전체적으로 '약함'으로 분류되었지만, 실제 이상 탐지 모델 학습 시 중요도가 높게 나올 수도 있다.


앞서 살펴본 것처럼, 단일 피처만으로 탐지되지 않던 거래들도 여러 기준에서 동시에 벗어날 경우 사기 가능성이 높아지게 된다.

# 결론

> _"사기 거래는 정상 거래와 어떤 행동 패턴이 다르고,
이 차이를 수치적로 설명할 수 있을까? 모델링 시 이 패턴을 신뢰할 수 있을까?"_
>

분석을 수행하면서 궁금증에 대한 답을 내려볼 수 있었다.

- 사기 거래는 단일 피처 수준에서도 충분히 이상 신호를 보이는 경우가 많았고, 특히 `amt_z`, `cat_amt_z`, `hour_perc`는 각각의 조건만으로도 충분한 설명력을 보였다.
이들이 결합되면, 단일 기준을 넘어서는 강한 이상 패턴으로 드러나는 경우도 확인할 수 있다.
- 반대로 개별 통계 차이가 작아 보이는 피처도 다른 변수와의 조합 속에서 충분히 의미를 가질 수 있음을 확인할 수 있다.

<br>
결과적으로

이번 프로젝트는 단순히 거래가 사기인지 아닌지를 분류하는 단계에서 나아가, 사기 거래가 어떤 방식으로 정상 거래의 행동 패턴에서 벗어나는지를 정량적으로 해석해 보는 경험이 되었다.

이 과정을 통해 도출된 피처들은 모델링에 앞서 신뢰할 수 있는 데이터를 마련해 주며, 이후 예측 모델이 실제 상황에서도 효과적으로 작동할 수 있는 기반이 될 것이라 본다.